I-43-16 Feynamn Lectures on Physics

https://www.feynmanlectures.caltech.edu/I_43.html



In [1]:
using DynamicPolynomials
using JuMP
using MosekTools
using LinearAlgebra
using CSV
using DataFrames
import JSON
using Dates
using Gurobi
using SymbolicUtils # for symbolic simplification

functions for generating monomials

In [2]:
function all_monomials_up_to_max_deg(x, deg)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:deg 
      for m=all_monomials_up_to_max_deg(x[2:end], deg)
    ]
    end
end

function mons_of_max_degree_and_unit(x, deg, u)
    [m
        for m=all_monomials_up_to_max_deg(x, deg)
        #if all(unit(m) .== u)
    ]
end
                
function degree_poly(p)
    maximum(degree.(monomials(p)))
end

function all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall) 
                for m=all_monomials_up_to_max_deg_overall(x[2:end], deg, deg_overall-k)
    ]
    end
end

function mons_of_max_degree_and_unit_overall(x, deg, deg_overall, u)
    [m
        for m=all_monomials_up_to_max_deg_overall(x, deg, deg_overall)
        #if all(unit(m) .== u)
    ]
end

function all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    if size(x,1) == 0
        [1]
    else
    [ x[1]^k * m for k=0:min(deg, deg_overall, theDegrees[1]) 
                for m=all_monomials_up_to_max_deg_overall_and_individual(x[2:end], deg, deg_overall-k, theDegrees[2:end])
    ]
    end
end

all_monomials_up_to_max_deg_overall_and_individual (generic function with 1 method)

In [3]:
@polyvar v m F tau mu tau qc E d Volt    # 

x   =   [v m F tau mu tau qc E d Volt ] #

1×10 Matrix{Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}}}:
 v  m  F  tau  mu  tau  q  E  d  Volt

In [4]:
axioms= 
[
    v * m - F * tau, 
    mu * m - tau, 
    F - qc * E, 
    E * d - Volt,  
]


4-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 -Ftau + vm
 -tau + mmu
 F - qE
 -Volt + Ed

In [6]:
deg = 3
deg_overall = 12
theDegrees = [ 2 2 2  2  2   2  2 2 2  2] 
             # v m F tau mu tau q E d Volt ] # 

candidate_mons = [
    #mons_of_max_degree_and_unit_overall(params, deg, deg_overall, [])
    all_monomials_up_to_max_deg_overall_and_individual(x, deg, deg_overall, theDegrees)
    for ai=axioms
]

@show size.(candidate_mons)

#model = Model(Mosek.Optimizer)
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "TimeLimit", 600.0)

mons_q = mons_of_max_degree_and_unit_overall([v  mu qc d  Volt ], deg, deg_overall, []) #  only include variables expected to appear in the final theorem 
coeff_q =   @variable(model, [1:size(mons_q,1)], base_name="q")

q = sum(ci .* mi for (ci, mi)=zip(coeff_q, mons_q)) # Zip pairs things without needing a ref index, e.g., zip([1, 2, 3], [4,5,6])=((1,4), (2,5), (3,6))
coeff_αs = [
    @variable(model, [1:size(X,1)], base_name="α$i")
    for (i,X)=enumerate(candidate_mons)
    ]
@show size.(coeff_αs)
αs = [sum(ci .* mi) for (ci, mi)=zip(coeff_αs, candidate_mons)]

residual = q - sum(αᵢ * aᵢ for (αᵢ, aᵢ)=zip(αs,axioms));
eqs = coefficients(residual)

# Ensure that the sum of the coefficients on the terms involving m isn't zero, in order that m is part of expression
@constraint model sum(coeff_q[degree.(mons_q, [v]).>0]) == 1.0

@constraint model eqs .== 0
@objective model Max 0

optimize!(model)
@show termination_status(model)

In [7]:
value_poly = p -> sum(value.(coefficients(p)).* monomials(p))

#43 (generic function with 1 method)

In [8]:
value_q = value_poly(q)
value_q

-p³c² + p³v² + m_0pv²c²

In [9]:
q_sim = simplify(value_q)

-p³c² + p³v² + m_0pv²c²

which can be reorganized to arrive at the known relation p = m_0 * v /sqrt(1 - v*v/c*c)

In [10]:
@show value_αs = value_poly.(αs)

value_αs = value_poly.(αs) = Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}[-p²c² + p²v² - mpvc² + mpv³, -pv²]


2-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Float64}}:
 -p²c² + p²v² - mpvc² + mpv³
 -pv²